In [15]:

import inspect
import itertools
import logging

logger = logging.getLogger(__name__)
logger.addHandler(logging.NullHandler())

class State:
    def __init__(self, name: str, on_enter:(str or list) =None, on_exit:(str or list) =None,
                 ignore_invalid_triggers: bool =False):
        
        self.name = name
        self.on_enter = listify(on_enter) if on_enter else []
        self.on_exit = listify(on_exit) if on_exit else []
        self.ignore_invalid_triggers = ignore_invalid_triggers
    
    def __str__(self):
        return str(self.name) + "*/*" + str(self.ignore_invalid_triggers) + "*/*" + str(self.on_enter) + "*/*" +\
                str(self.on_exit)
    
    def enter(self, event_data: EventData):
        """ Triggered when a state is entered. """
        logger.debug("%sEntering state %s. Processing callbacks...", event_data.machine.id, self.name)
        for oe in self.on_enter:
            event_data.machine._callback(oe, event_data)
        logger.info("%sEntered state %s", event_data.machine.id, self.name)

    def exit(self, event_data: EventData):
        """ Triggered when a state is exited. """
        logger.debug("%sExiting state %s. Processing callbacks...", event_data.machine.id, self.name)
        for oe in self.on_exit:
            event_data.machine._callback(oe, event_data)
        logger.info("%sExited state %s", event_data.machine.id, self.name)

    def add_callback(self, trigger: str, func: str):
        """ Add a new enter or exit callback."""
        callback_list = getattr(self, 'on_' + trigger)
        callback_list.append(func)
    
if __name__ == '__main__':
    s = State("drink")
    print(s)


drink*/*False*/*[]*/*[]


In [5]:


class Condition:
    pass

In [2]:


class Transition:
    pass

In [3]:


class EventData:
    pass

In [4]:


class Event:
    pass

In [ ]:


class Machine:
    pass